In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
from faker import Faker
from matplotlib import pyplot as plt

Reading the raw dataset.

In [ ]:
df_political = pd.read_csv(
    open(Path("../raw/polibertweet/published_data_polibertweet-LREC-2022_election_sampled_10000.csv")),
    encoding="utf-8"
)
df_political

In [ ]:
df_non_political = pd.read_csv(
    open(Path("../raw/polibertweet/published_data_polibertweet-LREC-2022_non_election_sampled_10000.csv")),
    encoding="utf-8"
)
df_non_political

Adding the politicalness label column.

In [ ]:
df_political["politicalness"] = ["political"] * len(df_political)
df_non_political["politicalness"] = ["non-political"] * len(df_non_political)
df = pd.concat([df_political, df_non_political])
df["politicalness"] = df["politicalness"].astype("category")

Renaming columns.

In [ ]:
df = df.rename(columns={"text": "body"})

Stripping the bodies.

In [ ]:
df["body"] = (df["body"].str.strip()).replace("", np.nan)

Dropping rows with missing data.

In [ ]:
df = df.dropna()

Printing body duplicates.

In [ ]:
df[df["body"].duplicated(keep=False)]

Dropping the body duplicates.

In [ ]:
df = df.drop_duplicates(subset="body")

Filling username and URL placeholders.

In [ ]:
fake = Faker()
fake.seed_instance(37)
df["body"] = df["body"].str.replace("@USER", lambda _: fake.name(), regex=True)
df["body"] = df["body"].str.replace("HTTPURL", lambda _: f"{fake.url()}{fake.uri_path()}", regex=True)
df

Inspecting the body length.

In [ ]:
df["body_length"] = df["body"].fillna("").str.len()
df["body_word_count"] = df["body"].fillna("").str.split().str.len()
df = df.sort_values(by="body_length")
df.head()

In [ ]:
# Ensuring to include the last (longest) item.
downsampled = pd.concat([df["body_length"].iloc[::100], df["body_length"].tail(1)]).drop_duplicates()
downsampled.plot.bar().xaxis.set_ticks([]);

In [ ]:
df["body_word_count"].median()

The politicalness distribution.

In [ ]:
df.groupby("politicalness", observed=True).size().plot.pie(autopct="%1.1f%%");

The distribution of body length sums per politicalness.

In [ ]:
df.groupby("politicalness", observed=True)["body_length"].sum().plot.pie(autopct="%1.1f%%", ylabel="");

The body length distribution by politicalness.

In [ ]:
plt.figure(figsize=(10, 6))

for politicalness in df["politicalness"].unique():
    df_politicalness = df[df["politicalness"] == politicalness]
    downsampled = pd.concat(
        # Ensuring to include the last (longest) item.
        [df_politicalness["body_length"].iloc[::10], df_politicalness["body_length"].tail(1)]
    ).drop_duplicates().reset_index(drop=True)
    plt.plot(downsampled, label=politicalness)

plt.xlabel("downsampled index")
plt.ylabel("body length")
plt.title("body length by politicalness")
plt.legend(title="politicalness")
plt.show()

In [ ]:
df.to_parquet("../preprocessed/polibertweet.parquet")